In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
maxDt <- (indices$BseConstituents() %>%
          filter(NAME == "sp bse sensex") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$BseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "sp bse sensex") %>%
     print(n = 30)

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
   NAME          TIME_STAMP   CODE SYMBOL     SECURITY_NAME                    
   <chr>         <date>      <int> <chr>      <chr>                            
 1 sp bse sensex 2019-07-01 500010 HDFC       Housing Development Finance Corp 
 2 sp bse sensex 2019-07-01 500034 BAJFINANCE Bajaj Finance Ltd                
 3 sp bse sensex 2019-07-01 500112 SBIN       State Bank of India              
 4 sp bse sensex 2019-07-01 500180 HDFCBANK   HDFC Bank Ltd                    
 5 sp bse sensex 2019-07-01 500182 HEROMOTOCO Hero MotoCorp Ltd                
 6 sp bse sensex 2019-07-01 500209 INFY       Infosys Ltd                      
 7 sp bse sensex 2019-07-01 500247 KOTAKBANK  Kotak Mahindra Bank Ltd          
 8 sp bse sensex 2019-07-01 500295 VEDL       Vedanta Ltd                      
 9 sp bse sensex 2019-07-01 500312 ONGC       Oil & Natural Gas Corp Ltd       
10 sp bse sensex 2019-0

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)